# Similaridade de texto (palavras)

A similaridade de texto é utilizada para comparar palavras similares (que foram escritas de forma diferente).
Alguns exemplos:

* `São Paulo`
* `Sao Paulo`
*  `São Paulo   ` (tem espaço no final)
* `são paulo`

Todas as palavras acima se referem a cidade de São Paulo, mas se efetuar umas simples comparação de _strings_ (`str1 == str2`) o resultado será `False`.

É possível tratar esses casos, como (`str.strip()`) para remover espaços no começo/final do texto.
Outro detalhe é `str.lower()` ou `str.upper()` para manter tudo minúsculo/maiúsculo.
Finalmente, é possível remover toda a acentuação para evitar problemas.

Entretanto, essa não é uma boa adordagem (principalmente remover acentuação, as outras duas podem ser boas ideias) visto que estamos alterando o texto para encontrar o nome `São Paulo`.

Uma alternativa é computar um número (entre 0 e 100, por exemplo) que indica o quão similares são duas palavras.
Sendo que, 0 indica palavras completamente diferentes e 100 indica que as palavras são idênticas.
Assim, podemos definir um limiar de forma que, qualquer similaridade acima de 80 aceitamos como algo válido.
O valor de limiar vai variar com o contexto, por isso é importante testar várias abordagens.

Existem várias métricas para calcular essa similaridade: https://en.wikipedia.org/wiki/Edit_distance

Vamos trabalhar com a distância de Levshtein: https://en.wikipedia.org/wiki/Levenshtein_distance

A distância de Levshtein calcula o número minímino de alterações (inserir, remover, substituir) que devemos fazer em uma string para que ela fique igual a outra.

Tutorial `fuzzywuzzy` https://www.datacamp.com/community/tutorials/fuzzy-string-python

In [4]:
from fuzzywuzzy import fuzz

/home/julio/Projects/pos-furb-recuperacao/.venv/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
fuzz.ratio('São Paulo', 'Sao Paulo')

89

In [6]:
fuzz.ratio('São Paulo', 'sao paulo')

67

In [7]:
fuzz.ratio('São Paulo', 'São Paulo    ')

82

In [8]:
fuzz.ratio('Sao Paulo', 'São Paulo')

89

Assim, podemos ter uma lista de cidades e ver quais pessoas moram em `São Paulo`

In [10]:
pessoas = [
    {'nome': 'Pessoa A', 'cidade': 'São Paulo'},
    {'nome': 'Pessoa B', 'cidade': ' São Paulo '},
    {'nome': 'Pessoa C', 'cidade': 'Sao Paulo'},
    {'nome': 'Pessoa D', 'cidade': 'sao paulo'},
    {'nome': 'Pessoa E', 'cidade': 'SaoPaulo'},
    {'nome': 'Pessoa F', 'cidade': 'S. Paulo'},
    {'nome': 'Pessoa G', 'cidade': 'Blumenau'},
    {'nome': 'Pessoa H', 'cidade': 'Pomerode'},
    {'nome': 'Pessoa I', 'cidade': 'Rio de Janeiro'},
]

scores = [fuzz.ratio('São Paulo', p['cidade']) for p in pessoas]
scores

[100, 90, 89, 67, 82, 82, 24, 24, 35]

Entretanto, nem sempre é possível comparar o texto diretamente.
Por exemplo, o nome da cidade pode estar no meio de uma frase.
Para isso, é possível fazer uma comparação parcial.

`fuzz.ratio()` compara dois textos por completo, com a similaridade entre eles.

`fuzz.partial_ratio()` faz a comparação em substrings.

In [13]:
fuzz.ratio('São Paulo', 'João mudou-se para São Paulo em 2019')

40

In [11]:
fuzz.partial_ratio('São Paulo', 'João mudou-se para São Paulo em 2019')

100

In [12]:
fuzz.partial_ratio('São Paulo', 'João mudou-se para Sao Paulo em 2019')

89

Outro caso comum, é a ordem dos textos.

Por exemplo, `Comparamos o Dispositivo A com o Dispositivo B` e `Comparamos o Dispositivo B com o Dispositivo A`.

In [14]:
fuzz.partial_ratio('Dispositivo A com Dispositivo B', 'Comparamos Dispositivo A com Dispositivo B')

100

In [17]:
fuzz.partial_ratio('Dispositivo A com Dispositivo B', 'Comparamos dispositivo B com dispositivo A')

55

In [18]:
fuzz.token_sort_ratio('Dispositivo A com Dispositivo B', 'Comparamos dispositivo B com dispositivo A')

85

Outro caso é o uso da comparação `Dispositivo A com Dispositivo B` ou `Dispositivo A x Dispositivo B`.

In [19]:
fuzz.token_sort_ratio('Dispositivo A x Dispositivo B', 'Comparamos dispositivo B com dispositivo A')

76

In [21]:
fuzz.token_set_ratio('Dispositivo A x Dispositivo B', 'Comparamos dispositivo B com dispositivo A')

94

## Processamento de linguagem natural

São técnicas utilizadas para trabalhar com texto que englobam desde _tokenização_ (separar uma string em palavras) até a tradução de texto.

A similaridade de palavras acima demonstra um uso de processamento de linguagem natural.

Nesta parte, vamos ver alguns conceitos básicos utilizando as bibliotecas `NLTK` e `spaCy`.

Uma comparação entre `NLTK` e `spaCy` https://medium.com/@akankshamalhotra24/introduction-to-libraries-of-nlp-in-python-nltk-vs-spacy-42d7b2f128f2

In [4]:
# Fonte http://www.furb.br/web/1704/noticias/furb-esta-entre-4-melhores-de-sc-em-ranking-da-america-latina/7944
texto = '''O Brasil desponta com as melhores Universidades da América Latina, com o total de 3 no top 10,
seguida do Chile, Colômbia e México com 2 em cada país, além da Argentina com uma.
No primeiro lugar está a Pontifícia Universidade Católica do Chile, seguida da Universidade de São Paulo (USP).
'''

In [1]:
import spacy

In [2]:
# pode ser necessário instalar o modelo
# python -m spacy download pt_core_news_sm
nlp = spacy.load('pt_core_news_sm')

In [5]:
doc = nlp(texto)

In [9]:
tokens = [f'"{token}"' for token in doc]
', '.join(tokens)

'"O", "Brasil", "desponta", "com", "as", "melhores", "Universidades", "da", "América", "Latina", ",", "com", "o", "total", "de", "3", "no", "top", "10", ",", "\n", "seguida", "do", "Chile", ",", "Colômbia", "e", "México", "com", "2", "em", "cada", "país", ",", "além", "da", "Argentina", "com", "uma", ".", "\n", "No", "primeiro", "lugar", "está", "a", "Pontifícia", "Universidade", "Católica", "do", "Chile", ",", "seguida", "da", "Universidade", "de", "São", "Paulo", "(", "USP", ")", ".", "\n"'

In [11]:
for (i, token) in enumerate(doc):
    print(token.text, token.pos_)
    if i == 10:
        break

O DET
Brasil PROPN
desponta VERB
com ADP
as DET
melhores ADJ
Universidades PROPN
da ADP
América PROPN
Latina PROPN
, PUNCT


In [13]:
# apenas os verbos
verbos = [token.text for token in doc if token.pos_ == 'VERB']
verbos

['desponta', 'top', 'seguida', 'está', 'seguida']

In [16]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Brasil LOC
Universidades da América Latina PER
Chile LOC
Colômbia LOC
México LOC
Argentina LOC
Pontifícia Universidade Católica do Chile LOC
Universidade de São Paulo LOC
USP LOC
